In [18]:
from matplotlib import pyplot as plt
from IPython.core.interactiveshell import InteractiveShell

import warnings
import numpy as np
import pandas as pd
import yfinance as yf

In [19]:
plt.rc('font', family='GULIM')
warnings.filterwarnings(action='ignore')
InteractiveShell.ast_node_interactivity = "all"

# Data Collecting

In [20]:
CL = yf.download('CL=F', start='2000-08-23')
HO = yf.download('HO=F', start='2000-09-01')
NG = yf.download('NG=F', start='2000-08-30')
RB = yf.download('RB=F', start='2000-11-01')
BZ = yf.download('BZ=F', start='2007-07-30')
ZL = yf.download('ZL=F', start='2000-03-15')
SP = pd.read_csv('Database/oil_price/CL_spot.csv', index_col=0)
BZ_2 = pd.read_csv('Database/oil_price/BZ_future.csv', index_col=0)
SP.sort_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


# Numerical Data Preprocessing

In [21]:
SP['price'] = pd.to_numeric(SP['price'], errors='coerce')
SP = SP.dropna()

In [22]:
BZ_2.index=pd.to_datetime(BZ_2.index)
BZ_2_sorted = BZ_2.sort_index().loc[:'2007-07-27', 'Price']
BL = pd.concat([BZ['Adj Close'], BZ_2_sorted], axis=0).sort_index()
BL=pd.DataFrame(data=BL.values, columns=['Adj Close'], index=BL.index)

In [23]:
CL['return'] = CL['Adj Close'].pct_change().apply(lambda x: np.log(1 + x))
HO['return'] = HO['Adj Close'].pct_change().apply(lambda x: np.log(1 + x))
NG['return'] = NG['Adj Close'].pct_change().apply(lambda x: np.log(1 + x))
RB['return'] = RB['Adj Close'].pct_change().apply(lambda x: np.log(1 + x))
BL['return'] = BL['Adj Close'].pct_change().apply(lambda x: np.log(1 + x))
ZL['return'] = ZL['Adj Close'].pct_change().apply(lambda x: np.log(1 + x))
SP['return'] = SP['price'].pct_change().apply(lambda x: np.log(1 + x))

In [24]:
CL = CL.dropna()
HO = HO.dropna()
NG = NG.dropna()
RB = RB.dropna()
BL = BL.dropna()
ZL = ZL.dropna()
SP = SP.dropna()

In [25]:
CL_vol = CL['return'].rolling(window=5).std()
HO_vol = HO['return'].rolling(window=5).std()
NG_vol = NG['return'].rolling(window=5).std()
RB_vol = RB['return'].rolling(window=5).std()
BZ_vol = BL['return'].rolling(window=5).std()
ZL_vol = ZL['return'].rolling(window=5).std()
SP_vol = SP['return'].rolling(window=5).std()

In [26]:
CL_vol = CL_vol.dropna()
HO_vol = HO_vol.dropna()
NG_vol = NG_vol.dropna()
RB_vol = RB_vol.dropna()
BZ_vol = BZ_vol.dropna()
ZL_vol = ZL_vol.dropna()
SP_vol = SP_vol.dropna()

In [28]:
CL_vol.index = pd.to_datetime(CL_vol.index)
HO_vol.index = pd.to_datetime(HO_vol.index)
NG_vol.index = pd.to_datetime(NG_vol.index)
RB_vol.index = pd.to_datetime(RB_vol.index)
BZ_vol.index = pd.to_datetime(BZ_vol.index)
ZL_vol.index = pd.to_datetime(ZL_vol.index)
SP_vol.index = pd.to_datetime(SP_vol.index)
vol_list = [CL_vol, HO_vol, NG_vol, RB_vol, BZ_vol, ZL_vol, SP_vol]

In [11]:
# plt.figure(figsize=(10, 6))
# # plt.plot(CL_vol[:500], label='CL')
# plt.plot(HO_vol[:600], label='HO')
# # plt.plot(NG_vol['return'][:500], label='NG')
# # plt.plot(RB_vol['return'][:500], label='RB')
# plt.plot(BZ_vol[:600], label='BZ')
# # plt.plot(ZL_vol['return'][:500], label='ZL')
# plt.title('Volatility')
# plt.xlabel('Time')
# plt.ylabel('Value')
# plt.legend()
# plt.xticks(CL_vol.index[0:600:50])
# plt.tight_layout()
# plt.show()

# Text Data Preprocessing

In [12]:
text_data=pd.read_parquet('Database/newsdata+embedding.parquet')

In [13]:
text_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27971 entries, 0 to 27970
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   title      27971 non-null  object        
 1   url        27971 non-null  object        
 2   date       27971 non-null  datetime64[ns]
 3   embedding  27971 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 874.2+ KB


In [14]:
NLP_vec = text_data[['date', 'embedding']].reset_index()
NLP_vec.index = NLP_vec['date']
NLP_vec = NLP_vec[~NLP_vec.index.duplicated(keep='first')]
NLP_vec = NLP_vec['embedding']

In [15]:
new_columns = {f'{i}': pd.Series(dtype='object') for i in range(384)}
new_df = pd.DataFrame(new_columns)

for i in range(384):
    new_df[f'{i}'] = NLP_vec.apply(lambda x: f'{x[i]}')

df = pd.concat([NLP_vec , new_df], axis=1)

In [16]:
vol_list = [CL_vol, HO_vol, NG_vol, RB_vol, BZ_vol, ZL_vol, SP_vol]
for i, vol in enumerate(vol_list):
    vol_list[i] = pd.concat([vol, df], axis=1)
    vol_list[i] = vol_list[i].dropna(subset=['return'])
    nan_index = vol_list[i][vol_list[i]['embedding'].isna()].index
    vol_list[i].loc[nan_index] = 0
    vol_list[i]=vol_list[i].drop(columns=['embedding'])
    if i !=6:
        vol_list[i]=vol_list[i].loc['2001-01-01':]
    else:
        vol_list[i]=vol_list[i].loc[:'2000-12-31']

# Save Files

In [29]:
name_list = ['CL_vol', 'HO_vol', 'NG_vol', 'RB_vol', 'BZ_vol', 'ZL_vol', 'SP_vol']
for i, vol in enumerate(name_list):
    if i != 6:
        vol_list[i].to_csv(f'Database/future_std/{vol}.csv')
    else:
        vol_list[i].to_csv(f'Database/spot_std/{vol}.csv')